In [1]:
import torch
import random
import datetime
import pandas as pd
import numpy as np
import os

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, fit_treshold
from src.embeddings import RecsysEmbedding
from sklearn.linear_model import LogisticRegression

experiment_name = 'NeuralClickModel'
device = 'cuda:2'
seed = 7331
pkl_path = '../data/'


random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
x = torch.arange(60).reshape(4,3,5)
print(x, x.flatten(0,1))

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14]],

        [[15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]],

        [[30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44]],

        [[45, 46, 47, 48, 49],
         [50, 51, 52, 53, 54],
         [55, 56, 57, 58, 59]]]) tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]])


In [3]:
def flatten(true, pred, mask, to_cpu=True):
    mask = mask.flatten()
    nnz_idx = mask.nonzero()[:, 0]
    true, pred = [x.flatten()[nnz_idx] for x in [true, pred]]
    if to_cpu:
        true, pred = [x.cpu().numpy() for x in [true, pred]]
    return true, pred


# Модель

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class NeuralClickModel(nn.Module):
    def __init__(self, embedding, readout=False):
        super().__init__()
        
        self.embedding = embedding
        
        self.emb_dim = embedding.embedding_dim
        self.rnn_layer = nn.GRU(
            input_size=self.emb_dim * 2, 
            hidden_size=self.emb_dim, 
            batch_first=True
        )
        self.out_layer = nn.Linear(self.emb_dim, 1)
        
        self.thr = -1.5
        self.readout = readout
        self.readout_mode = 'threshold' # ['soft' ,'threshold', 'sample', 'diff_sample']
        
        self.calibration = False
        self.w = 1
        self.b = 0

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        max_sequence = item_embs.size(1)
#         print({k:v.shape for k, v in batch.items()})
#         print(item_embs.shape, user_embs.shape)
        # ilya format
        # 'items': (batch, slate, 2*embedding_dim ) 2, нужно для readout, по умолчанию ноли на половине эмбеддинга
        # 'clicks': (batch, slate)
        # 'users': (1, batch, embedding_dim), 
        # 'mask': (batch, slate)
        
        x = {}
        x['items'] = torch.cat(
            [
                item_embs.flatten(0,1),
                torch.zeros_like(item_embs.flatten(0,1)),
            ],
            dim = -1
        )
                
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2).unsqueeze(0)
        x['users'] = user_embs.repeat_interleave(max_sequence, 1)
        x['clicks'] = (batch['responses'].flatten(0,1) > 0 ).int().clone()
        x['mask'] = batch['slates_mask'].flatten(0,1).clone()
        
#         print(*[(k,v) for k,v in x.items()], sep='\n')
                
#         items = F.dropout(x['items'], p=0.1, training=self.training)
#         h = F.dropout(x['users'], p=0.1, training=self.training)
        items = x['items']
        h = x['users']
        
        if self.readout:
            res = []
            seq_len = items.shape[1]
            for i in range(seq_len):
#                 print(items[:,[i],:])
                output, h = self.rnn_layer(items[:,[i],:], h)
                y = self.out_layer(output)[:, :, 0]
                
                # readout
                if i + 1 < seq_len:
                    if self.readout_mode == 'threshold':
                        items[:, [i+1], self.emb_dim:] *= (y.detach()[:, :, None] > self.thr).to(torch.float32)
                    elif self.readout_mode == 'soft':
                        items[:, [i+1], self.emb_dim:] *= torch.sigmoid(y)[:, :, None]
                    elif self.readout_mode == 'diff_sample' or self.readout_mode == 'sample':
                        eps = 1e-8
                        gumbel_sample = -( (torch.rand_like(y) + eps).log() / (torch.rand_like(y) + eps).log() + eps).log()
                        T = 0.3
                        bernoulli_sample = torch.sigmoid( (nn.LogSigmoid()(self.w * y + self.b) + gumbel_sample) / T )
                        if self.readout_mode == 'sample':
                            bernoulli_sample = bernoulli_sample.detach()
                        items[:, [i+1], self.emb_dim:] *= bernoulli_sample[:, :, None]
                    else:
                        raise
                    
                res.append(y)
        
            y = torch.cat(res, axis=1)
            
        else:
            items[:, 1:, self.emb_dim:] *= x['clicks'][:, :-1, None]
#             print(items)
            rnn_out, _ = self.rnn_layer(items, h)
            y = self.out_layer(rnn_out)[:, :, 0]
        
        
        if self.calibration and self.training:
            clicks_flat, logits_flat = flatten(x['clicks'], y.detach(), x['mask'])
            # print(clicks_flat.mean(), torch.sigmoid(torch.Tensor([-1.5])))
            logreg = LogisticRegression()
#             print(logits_flat[:, None], clicks_flat)
            logreg.fit(logits_flat[:, None], clicks_flat)
            γ = 0.3
            self.w = (1 - γ) * self.w + γ * logreg.coef_[0, 0]
            self.b = (1 - γ) * self.b + γ * logreg.intercept_[0]
            y = self.w * y + self.b
        else:
            y = self.w * y + self.b
            
        return y.reshape(shp[:-1])
    

d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)
emb = RecsysEmbedding(d.n_items, dummy_matrix, embeddings='explicit')

for batch in dummy_loader:
    break

    
model = NeuralClickModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='svd', embedding_dim=2),
    readout=True
).to('cpu')

model(batch)


3it [00:00, 3313.91it/s]

biulding affinity matrix...



/opt/conda/lib/python3.7/site-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


tensor([[[-0.2637, -0.0412,  0.1598],
         [-0.1717,  0.0120,  0.0835]],

        [[-0.3949, -0.4857, -0.1529],
         [-0.0378,  0.0587,  0.1066]]], grad_fn=<ViewBackward>)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [5]:
# d = DummyData()
# dummy_loader, dummy_svd = get_dummy_data(d)

# print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
# model = NeuralClickModel(
#     RecsysEmbedding(d.n_items, dummy_matrix, embeddings='svd', embedding_dim = 2),
#     readout=False
# ).to('cpu')

# _, metrics = train(model, 
#    dummy_loader, dummy_loader, dummy_loader,
#    device='cpu', lr=1e-3, num_epochs=5000, 
#    silent=True,
# #    debug=True
# )


# ContentWise

In [6]:
content_wise_results = []
c = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
c_train_loader, c_val_loader, c_test_loader, c_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(c, batch_size=150)
len(c_train_loader), len(c)

(108, 20216)

In [7]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = NeuralClickModel(
        RecsysEmbedding(train_num_items, c_train_user_item_matrix, embeddings='neural').to('cpu'),
    ).to(device)

    _, metrics = train(model, 
       c_train_loader, c_val_loader, c_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)
    
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')


Evaluating NeuralClickModel with svd embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.17659352719783783, 'roc-auc': 0.4901577830314636, 'accuracy': 0.2116405963897705}


train... loss:49.738331228494644:   0%|                                                                           | 1/5000 [00:21<30:08:26, 21.71s/it]

Test update: epoch: 0 |accuracy: 0.21673116087913513 | f1: 0.1895570158958435 | auc: 0.6129519939422607 | treshold: 0.06999999999999999


...

train... loss:28.544926211237907:   0%|▏                                                                          | 9/5000 [01:24<13:00:31,  9.38s/it]

Test update: epoch: 8 |accuracy: 0.8802542090415955 | f1: 0.26655852794647217 | auc: 0.7245150804519653 | treshold: 0.18000000000000002


train... loss:28.12286275625229:   0%|▏                                                                          | 10/5000 [01:33<13:02:57,  9.41s/it]

Test update: epoch: 9 |accuracy: 0.8704997897148132 | f1: 0.29369282722473145 | auc: 0.723408579826355 | treshold: 0.2


train... loss:24.78700192272663:   1%|▋                                                                          | 44/5000 [06:49<12:49:01,  9.31s/it]


# RL4RS

In [8]:
rl4rs_results = []
r = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
r_train_loader, r_val_loader, r_test_loader, r_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(r, batch_size=20000)
len(r_train_loader), len(r)

(2, 45942)

In [11]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = NeuralClickModel(
        RecsysEmbedding(train_num_items, r_train_user_item_matrix, embeddings='neural'),
    ).to(device)

    _, metrics = train(model, 
       r_train_loader, r_val_loader, r_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=50,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)
    
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')


Evaluating NeuralClickModel with neural embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.6724578142166138, 'roc-auc': 0.5068220496177673, 'accuracy': 0.5596179366111755}


train... loss:1.363502323627472:   0%|                                                                            | 1/5000 [00:35<49:12:04, 35.43s/it]

Test update: epoch: 0 |accuracy: 0.639680802822113 | f1: 0.7802504301071167 | auc: 0.5262739658355713 | treshold: 0.01


...

train... loss:0.5749185681343079:   7%|█████▎                                                                    | 360/5000 [31:12<7:08:08,  5.54s/it]

Test update: epoch: 359 |accuracy: 0.8429694175720215 | f1: 0.8752017617225647 | auc: 0.9152936339378357 | treshold: 0.49


train... loss:0.5712255239486694:   7%|█████▎                                                                    | 361/5000 [31:17<7:08:39,  5.54s/it]

Test update: epoch: 360 |accuracy: 0.8423165082931519 | f1: 0.8746371269226074 | auc: 0.9154651165008545 | treshold: 0.47000000000000003


train... loss:0.5809687972068787:   9%|██████▍                                                                   | 431/5000 [37:15<6:35:01,  5.19s/it]
